In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# initial DB of test samples including nuclide vectors
sfco_pkl = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
sfcoDB = pd.read_pickle(sfco_pkl)

# results DBs with LogLikelihoods and predictions
sfco_path = '~/sims_n_results/nucmoles_opusupdate_aug2019/sfco/'
uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
sfcoLL = {}
for i, unc_job in enumerate(job_dirs):
    sfcoLL[str(i)] = pd.read_csv(sfco_path + unc_job + '/' + unc_job + '.csv')

In [4]:
for unc_df in list(sfcoLL.keys()):
    df = sfcoLL[unc_df]
    df['Percent_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    #df['AdjDiff'] = df['MaxLogLL'] - df['2ndMaxLogLL']
    #df['X'] = df['AdjDiff'] / np.log(2)

In [5]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
ratios = sfcoDB.loc[:, lbls].copy()

#cs137/cs133
ratios['cs137/cs133'] = sfcoDB['cs137'] / sfcoDB['cs133']
#cs134/cs137
ratios['cs134/cs137'] = sfcoDB['cs134'] / sfcoDB['cs137']
#cs135/cs137
ratios['cs135/cs137'] = sfcoDB['cs135'] / sfcoDB['cs137']
#ba136/ba138
ratios['ba136/ba138'] = sfcoDB['ba136'] / sfcoDB['ba138']
#sm150/sm149
ratios['sm150/sm149'] = sfcoDB['sm150'] / sfcoDB['sm149']
#sm152/sm149
ratios['sm152/sm149'] = sfcoDB['sm152'] / sfcoDB['sm149']
#eu154/eu153
ratios['eu154/eu153'] = sfcoDB['eu154'] / sfcoDB['eu153']
#pu240/pu239
ratios['pu240/pu239'] = sfcoDB['pu240'] / sfcoDB['pu239']
#pu241/pu239
ratios['pu241/pu239'] = sfcoDB['pu241'] / sfcoDB['pu239']
#pu242/pu239
ratios['pu242/pu239'] = sfcoDB['pu242'] / sfcoDB['pu239']

ratios.replace([np.inf, -np.inf], 0, inplace=True)
ratios.fillna(0, inplace = True)

In [6]:
sfcoLL['0'].columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType', 'CoolingTime',
       'Enrichment', 'Burnup', 'OrigenReactor', 'pred_idx', 'pred_ReactorType',
       'pred_CoolingTime', 'pred_Enrichment', 'pred_Burnup',
       'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc', '2ndMaxLogLL',
       '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Percent_Burnup_Error'],
      dtype='object')

In [7]:
sfcoDB.columns

Index(['Burnup', 'CoolingTime', 'Enrichment', 'OrigenReactor', 'ReactorType',
       'ba136', 'ba138', 'cs133', 'cs134', 'cs135', 'cs137', 'eu153', 'eu154',
       'pu239', 'pu240', 'pu241', 'pu242', 'sm149', 'sm150', 'sm152'],
      dtype='object')

In [8]:
ratios.columns

Index(['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor',
       'cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
       'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
       'pu241/pu239', 'pu242/pu239'],
      dtype='object')

In [9]:
sfcoLL['0'].shape, ratios.shape

((544, 38), (544, 15))

## Explore high-burnup error cases

In [10]:
to_print = ['sim_idx', 'ReactorType', 'Enrichment', 'Burnup', 'pred_idx', 'pred_ReactorType', 'pred_Enrichment', 'pred_Burnup', 'ReactorType_Score', 'Enrichment_Error', 'Burnup_Error', 'Percent_Burnup_Error']
hundo_error = sfcoLL['0'].loc[sfcoLL['0']['Percent_Burnup_Error'] > 1.0, to_print]
hundo_error

,sim_idx,ReactorType,Enrichment,Burnup,pred_idx,pred_ReactorType,pred_Enrichment,pred_Burnup,ReactorType_Score,Enrichment_Error,Burnup_Error,Percent_Burnup_Error
36,FDN-2|2F2DN23|SF98|1,bwr,0.710,4150.0,11243,bwr,0.500,10045.28,True,0.210,5895.28,1.420549
59,FDN-2|2F2DN23|SF99|10,bwr,0.710,7190.0,46,pwr,0.530,15086.61,False,0.180,7896.61,1.098277
267,NPD-1|1022|A|1,phwr,0.711,791.0,20194,bwr,1.360,2961.22,False,0.649,2170.22,2.743641
268,NPD-1|1022|B|1,phwr,0.711,854.0,14605,bwr,2.160,2826.34,False,1.449,1972.34,2.309532
269,NPD-1|1022|C|1,phwr,0.711,1059.0,14605,bwr,2.160,2826.34,False,1.449,1767.34,1.668876
270,NPD-1|1129|A|1,phwr,0.711,1223.0,18736,bwr,1.620,3244.15,False,0.909,2021.15,1.652617
271,NPD-1|1129|B|1,phwr,0.711,1306.0,196,pwr,1.410,3609.51,False,0.699,2303.51,1.763790
272,NPD-1|1129|C|1,phwr,0.711,1622.0,17287,bwr,1.470,4174.70,False,0.759,2552.70,1.573798
273,NPD-1|1630|A|1,phwr,0.711,2733.0,17052,bwr,0.540,5915.89,False,0.171,3182.89,1.164614
274,NPD-1|1630|B|1,phwr,0.711,2733.0,24,pwr,0.530,6414.74,False,0.181,3681.74,1.347142


In [11]:
# worst Burnup pred: sim_idx is NPD-1|1022|A|1

# 922 is 180% error:
ratios.loc['NPD-1|922|A|1']

ReactorType                                             phwr
CoolingTime                                                0
Enrichment                                             0.711
Burnup                                                  3809
OrigenReactor    Nuclear Power Demonstration Reactor-1_CANDU
cs137/cs133                                                0
cs134/cs137                                                0
cs135/cs137                                                0
ba136/ba138                                                0
sm150/sm149                                                0
sm152/sm149                                                0
eu154/eu153                                                0
pu240/pu239                                         0.215892
pu241/pu239                                        0.0313633
pu242/pu239                                        0.0042531
Name: NPD-1|922|A|1, dtype: object

In [12]:
sfcoLL['0'].loc[sfcoLL['0']['sim_idx'] == 'NPD-1|922|A|1']

,Unnamed: 0,Unnamed: 0.1,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,...,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01,ReactorType_Score,CoolingTime_Error,Enrichment_Error,Burnup_Error,OrigenReactor_Score,Percent_Burnup_Error
276,6,6,NPD-1|922|A|1,phwr,0.0,0.711,3809.0,Nuclear Power Demonstration Reactor-1_CANDU,24135,phwr,...,-13286.870927,21.498057,-3.956407e+07,10.979891,True,1688.547527,0.0,7234.63,False,1.899352


In [13]:
ratios['pu240/pu239'].describe()

count    544.000000
mean       0.392998
std        0.202286
min        0.050855
25%        0.246832
50%        0.383785
75%        0.487914
max        1.001067
Name: pu240/pu239, dtype: float64

In [14]:
ratios['pu241/pu239'].describe()

count    544.000000
mean       0.173620
std        0.092511
min        0.000000
25%        0.105677
50%        0.184644
75%        0.249003
max        0.399240
Name: pu241/pu239, dtype: float64

In [15]:
ratios['pu242/pu239'].describe()

count    544.000000
mean       0.113038
std        0.156429
min        0.000107
25%        0.019027
50%        0.060537
75%        0.136886
max        1.039843
Name: pu242/pu239, dtype: float64

## Explore how many ratio entries are 0

In [16]:
# do not rerun
ratios['> zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].gt(0).sum(axis=1)
ratios['== zero'] = ratios[ratios.columns[~ratios.columns.isin(lbls)]].eq(0).sum(axis=1)

In [17]:
ratios.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
BAL-2|1476|42|15,pwr,0.0,4.4,45100.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425041,0.280066,0.113674,3,7
BAL-2|1476|42|31,pwr,0.0,4.4,45800.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428110,0.274637,0.113086,3,7
BAL-2|1476|42|6,pwr,0.0,4.4,46700.0,Balakovo-2_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.437705,0.288525,0.118033,3,7
BAL-3|1591|23|912,pwr,0.0,4.4,46200.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.418530,0.281150,0.119808,3,7
BAL-3|1591|23|581,pwr,0.0,4.4,47900.0,Balakovo-3_VVER-1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.440789,0.289474,0.144737,3,7


In [18]:
count = ratios.loc[ratios['> zero'] == 9]
count.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
DOD-1|Y013|UO2-b2|DU1,bwr,0.0,4.94,55490.0,Dodewaard-1_BWR,1.000000,0.012397,0.338602,0.0,181.385814,66.051661,0.113984,0.684829,0.213328,0.213328,9,1
GOS-1|1701|16B05|GU4 (Cycle 18),pwr,0.0,4.10,29070.0,Gösgen-1_PWR,0.919501,0.041622,0.375113,0.0,80.450630,31.348575,0.147344,0.356064,0.191205,0.059996,9,1
GOS-1|1701|16B05|GU3 (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.074341,0.081027,0.268415,0.0,96.816114,28.265107,0.198184,0.489219,0.289633,0.175091,9,1
GOS-1|1701|16B05|GU3' (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.143944,0.068570,0.254712,0.0,132.757594,39.874926,0.188847,0.487285,0.278694,0.174055,9,1
GOS-1|1240 (Cycle 15)|14H13|GU1 (Cycle 15),pwr,0.0,3.50,59660.0,Gösgen-1_PWR,1.178074,0.053422,0.254161,0.0,154.986276,50.503202,0.153502,0.649622,0.294743,0.317856,9,1


In [19]:
ratios['> zero'].describe()

count    544.000000
mean       3.867647
std        1.722172
min        2.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        9.000000
Name: > zero, dtype: float64

In [ ]:
ratios

## Explore 9 ratios cases

In [56]:
rat9 = ratios.loc[ratios['> zero'] == 9]
rat9.head()

,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,> zero,== zero
DOD-1|Y013|UO2-b2|DU1,bwr,0.0,4.94,55490.0,Dodewaard-1_BWR,1.000000,0.012397,0.338602,0.0,181.385814,66.051661,0.113984,0.684829,0.213328,0.213328,9,1
GOS-1|1701|16B05|GU4 (Cycle 18),pwr,0.0,4.10,29070.0,Gösgen-1_PWR,0.919501,0.041622,0.375113,0.0,80.450630,31.348575,0.147344,0.356064,0.191205,0.059996,9,1
GOS-1|1701|16B05|GU3 (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.074341,0.081027,0.268415,0.0,96.816114,28.265107,0.198184,0.489219,0.289633,0.175091,9,1
GOS-1|1701|16B05|GU3' (Cycle 18),pwr,0.0,4.10,52500.0,Gösgen-1_PWR,1.143944,0.068570,0.254712,0.0,132.757594,39.874926,0.188847,0.487285,0.278694,0.174055,9,1
GOS-1|1240 (Cycle 15)|14H13|GU1 (Cycle 15),pwr,0.0,3.50,59660.0,Gösgen-1_PWR,1.178074,0.053422,0.254161,0.0,154.986276,50.503202,0.153502,0.649622,0.294743,0.317856,9,1


In [21]:
rat9IsoLL = pd.merge(rat9, sfcoLL['0'], left_index=True, right_on='sim_idx')

In [22]:
rat9IsoLL.columns

Index(['ReactorType_x', 'CoolingTime_x', 'Enrichment_x', 'Burnup_x',
       'OrigenReactor_x', 'cs137/cs133', 'cs134/cs137', 'cs135/cs137',
       'ba136/ba138', 'sm150/sm149', 'sm152/sm149', 'eu154/eu153',
       'pu240/pu239', 'pu241/pu239', 'pu242/pu239', '> zero', '== zero',
       'Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType_y',
       'CoolingTime_y', 'Enrichment_y', 'Burnup_y', 'OrigenReactor_y',
       'pred_idx', 'pred_ReactorType', 'pred_CoolingTime', 'pred_Enrichment',
       'pred_Burnup', 'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc',
       '2ndMaxLogLL', '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Per

In [23]:
sfcoLL['0']['MaxLogLL'].describe()

count     544.000000
mean      -28.584146
std       321.242060
min     -7251.046687
25%       -20.345720
50%         9.934160
75%        12.193510
max        21.811218
Name: MaxLogLL, dtype: float64

In [24]:
rat9IsoLL['MaxLogLL'].describe()

count      39.000000
mean     -310.521263
std      1170.475640
min     -7251.046687
25%      -141.760022
50%       -83.842861
75%       -38.314910
max        -3.736192
Name: MaxLogLL, dtype: float64

## Explore highest LL predictions

Note the best predictions in terms of LL have burnup errors > 100% for PHWRs

In [25]:
to_print = ['sim_idx', 'ReactorType', 'Enrichment', 'Burnup', 'pred_idx', 'pred_ReactorType', 'pred_Enrichment', 'pred_Burnup', 'ReactorType_Score', 'Enrichment_Error', 'Burnup_Error', 'Percent_Burnup_Error', 'MaxLogLL']
bestLL = sfcoLL['0'].loc[sfcoLL['0']['MaxLogLL'] > 18, to_print]
bestLL

,sim_idx,ReactorType,Enrichment,Burnup,pred_idx,pred_ReactorType,pred_Enrichment,pred_Burnup,ReactorType_Score,Enrichment_Error,Burnup_Error,Percent_Burnup_Error,MaxLogLL
138,JPD-1|A-18|C3|-293,bwr,2.600,2907.0,21894,bwr,2.19,2934.73,True,0.410,27.73,0.009539,21.811218
139,JPD-1|A-14|C3|+538,bwr,2.600,3344.0,18006,bwr,4.35,3145.26,True,1.750,198.74,0.059432,20.445807
141,JPD-1|A-14|C3|-293,bwr,2.600,4085.0,6855,pwr,1.91,3987.23,False,0.690,97.77,0.023934,19.947932
143,JPD-1|A-18|C3|+049,bwr,2.600,4304.0,3615,pwr,2.15,4136.97,False,0.450,167.03,0.038808,19.901196
182,MIH-3|JPNNM3SFA1|E1|86B03,pwr,3.208,6900.0,23406,pwr,3.82,5670.58,True,0.612,1229.42,0.178177,20.833871
269,NPD-1|1022|C|1,phwr,0.711,1059.0,14605,bwr,2.16,2826.34,False,1.449,1767.34,1.668876,21.271682
270,NPD-1|1129|A|1,phwr,0.711,1223.0,18736,bwr,1.62,3244.15,False,0.909,2021.15,1.652617,21.770452
271,NPD-1|1129|B|1,phwr,0.711,1306.0,196,pwr,1.41,3609.51,False,0.699,2303.51,1.763790,21.222108
272,NPD-1|1129|C|1,phwr,0.711,1622.0,17287,bwr,1.47,4174.70,False,0.759,2552.70,1.573798,20.249597
273,NPD-1|1630|A|1,phwr,0.711,2733.0,17052,bwr,0.54,5915.89,False,0.171,3182.89,1.164614,18.430371


In [26]:
bestLL = sfcoLL['0'].loc[sfcoLL['0']['MaxLogLL'] > 18]
bestIsoLL = pd.merge(ratios, bestLL, left_index=True, right_on='sim_idx')

In [27]:
bestIsoLL.columns

Index(['ReactorType_x', 'CoolingTime_x', 'Enrichment_x', 'Burnup_x',
       'OrigenReactor_x', 'cs137/cs133', 'cs134/cs137', 'cs135/cs137',
       'ba136/ba138', 'sm150/sm149', 'sm152/sm149', 'eu154/eu153',
       'pu240/pu239', 'pu241/pu239', 'pu242/pu239', '> zero', '== zero',
       'Unnamed: 0', 'Unnamed: 0.1', 'sim_idx', 'ReactorType_y',
       'CoolingTime_y', 'Enrichment_y', 'Burnup_y', 'OrigenReactor_y',
       'pred_idx', 'pred_ReactorType', 'pred_CoolingTime', 'pred_Enrichment',
       'pred_Burnup', 'pred_OrigenReactor', 'MaxLogLL', 'MaxLLUnc',
       '2ndMaxLogLL', '2ndMaxLLUnc', 'CDF_LogLL_0.9998', 'CDF_LLUnc_0.9998',
       'CDF_LogLL_0.9988', 'CDF_LLUnc_0.9988', 'CDF_LogLL_0.95',
       'CDF_LLUnc_0.95', 'CDF_LogLL_0.9', 'CDF_LLUnc_0.9', 'CDF_LogLL_0.5',
       'CDF_LLUnc_0.5', 'CDF_LogLL_0.1', 'CDF_LLUnc_0.1', 'CDF_LogLL_0.01',
       'CDF_LLUnc_0.01', 'ReactorType_Score', 'CoolingTime_Error',
       'Enrichment_Error', 'Burnup_Error', 'OrigenReactor_Score',
       'Per

In [28]:
bestIsoLL[['> zero']]

,> zero
138,3
139,3
141,3
143,3
182,4
269,3
270,3
271,3
272,3
273,3


## Explore all available Isos in SFCOMPO, not just TAMU-filtered

In [36]:
allIsos = pd.read_pickle('~/sfcompo/format_clean/sfcompo_all_isos_formatted.pkl')
allIsos.head()

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,sm154,sr90,tc99,u,u232,u233,u234,u235,u236,u238
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.64,6.17,926.55
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.226,10.66,6.00,925.00
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.27,6.11,925.18
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,10.18,6.15,925.48
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,8.18,6.34,925.66


In [41]:
# do not rerun
allIsos['> zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].gt(0).sum(axis=1)
allIsos['== zero'] = allIsos[allIsos.columns[~allIsos.columns.isin(lbls)]].eq(0).sum(axis=1)
allIsos

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
BAL-2|1476|42|15,45100.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.048,0.0,...,0.0,0.0,0.0,0.0,0.000,10.640,6.170,926.55,17,68
BAL-2|1476|42|31,45800.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.226,10.660,6.000,925.00,13,72
BAL-2|1476|42|6,46700.0,0.0,4.4,Balakovo-2_VVER-1000,pwr,0.0,0.0,0.0,0.066,0.0,...,0.0,0.0,0.0,0.0,0.000,10.270,6.110,925.18,17,68
BAL-3|1591|23|912,46200.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.054,0.0,...,0.0,0.0,0.0,0.0,0.000,10.180,6.150,925.48,17,68
BAL-3|1591|23|581,47900.0,0.0,4.4,Balakovo-3_VVER-1000,pwr,0.0,0.0,0.0,0.071,0.0,...,0.0,0.0,0.0,0.0,0.000,8.180,6.340,925.66,17,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YNK-1|E6|NW-A1|G-127,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.111,7.249,4.770,932.06,10,75
YNK-1|E6|NW-A1|G-128,42500.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.105,7.125,4.598,932.39,10,75
YNK-1|E6|NW-A1|N-21,42600.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.102,7.373,4.569,931.37,8,77
YNK-1|E6|NW-A1|T-165,43200.0,0.0,3.4,Yankee-1_PWR,pwr,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.104,7.020,4.703,931.16,9,76


In [42]:
allIsos['> zero'].describe()

count    544.000000
mean      17.608456
std       11.805795
min        5.000000
25%        9.000000
50%       12.000000
75%       26.000000
max       66.000000
Name: > zero, dtype: float64

In [52]:
df = allIsos.loc[allIsos['> zero']<8]
df

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
GAR-1|SA-13|E6|11,4200.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,5,80
GAR-1|SA-13|E6|13,5580.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.00467,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|1,6080.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.00276,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|16,6640.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.03060,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|10,8140.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.00550,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|3,8260.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.01079,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|7,8320.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.01450,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
GAR-1|SA-13|E6|5,8640.0,0.0,2.410,Garigliano-1_BWR,bwr,0.0,0.0,0.0,0.02196,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00,6,79
NPD-1|1022|A|1,791.0,0.0,0.711,Nuclear Power Demonstration Reactor-1_CANDU,phwr,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,6.23490,0.0,992.41,7,78
NPD-1|1022|B|1,854.0,0.0,0.711,Nuclear Power Demonstration Reactor-1_CANDU,phwr,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,6.13010,0.0,992.41,6,79


In [54]:
df[df.iloc[:, 5:-1].gt(0.0, axis=1)]

,Burnup,CoolingTime,Enrichment,OrigenReactor,ReactorType,ag109,ag110m,am,am241,am242,...,tc99,u,u232,u233,u234,u235,u236,u238,> zero,== zero
GAR-1|SA-13|E6|11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
GAR-1|SA-13|E6|13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00467,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00276,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03060,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00550,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01079,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01450,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
GAR-1|SA-13|E6|5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02196,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN
NPD-1|1022|A|1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.23490,NaN,992.41,7,NaN
NPD-1|1022|B|1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.13010,NaN,992.41,6,NaN
